In [64]:
import pandas as pd
import numpy as np
import networkx as nx
#%matplotlib notebook
import matplotlib.pyplot as plt

import os
import operator
import warnings
#warnings.filterwarnings('ignore')
import plotly.graph_objects as go


In [65]:
cols_list=["Source airport","Destination airport"]
G_df = pd.read_csv('routes.csv',usecols=cols_list)#solo tiene 2 columnas por CODIGOS
#Count_df = pd.read_csv('D:/Personal and studies/College/Semester 6/Social and information networks project/countries.txt')
cols_list=["City","Country","IATA","Latitude","Longitude"]
airport_df = pd.read_csv('airports.csv',usecols=cols_list)#5 columnas 

In [66]:
G_df.head(10)

,Source airport,Destination airport
0,AER,KZN
1,ASF,KZN
2,ASF,MRV
3,CEK,KZN
4,CEK,OVB
5,DME,KZN
6,DME,NBC
7,DME,TGK
8,DME,UUA
9,EGO,KGD


In [67]:
#Count_df.head(2)

In [68]:
airport_df.head(2)

,City,Country,IATA,Latitude,Longitude
0,Goroka,Papua New Guinea,GKA,-6.08169,145.391998
1,Madang,Papua New Guinea,MAG,-5.20708,145.789001


In [69]:
G = nx.from_pandas_edgelist(G_df.head(1000), 'Source airport', 'Destination airport',create_using=nx.DiGraph())
#DiGraph itialize a graph with edges, name, or graph attributes
#This selects the first 10,000 rows of your DataFrame G_df. 'Source airport' and 'Destination airport':
# These are the column names in your DataFrame that represent the source and destination nodes for each edge

#Para el grafo mas basico
pos = nx.spring_layout(G)

# Draw the graph using Matplotlib
nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=10)
plt.show()

After running this code, you have a directed graph G created from the first 10,000 rows of your DataFrame, where nodes represent airports, and edges represent routes between them.

In [70]:
available_IATA=list(airport_df["IATA"])
nodes_in_G=list(G.nodes())#Converti los nodos en una lista 
#print(G.nodes())

In [71]:
#for i in nodes_in_G:
#    if i not in available_IATA:
#        G.remove_node(i)
#nodes_in_G=list(G.nodes())

In [72]:
#explicacion: set positions for nodes in your graph based on latitude and longitude information from the airport_df. 
# If a node in your graph has corresponding latitude and longitude values in the airport_df, you use those values as the 
# position for that node. If there is no match in airport_df, you remove the node from the graph
dict_pos={}
c=0#Sera el numero de nodos removidos 
for i in nodes_in_G:
    ## Find the corresponding row in airport_df based on 'IATA' column
    x=np.array(airport_df.loc[airport_df['IATA'].isin([i])][["Latitude","Longitude"]])
    #x contains the latitude and longitude values corresponding to the current node (i) 
    if len(x)==0:
        # If no match, remove the node from the graph!!
        G.remove_node(i)
        c=c+1
        continue
    else:
        # If a match is found, set the node's position based on latitude and longitude
        dict_pos[i]=x[0]#Diccionario con nodos y localizaciones 
print(c)

14


In [73]:
nx.set_node_attributes(G, dict_pos, 'pos')
#is setting node attributes in the NetworkX graph G. The attributes being set are positions ('pos') based on the latitude
# and longitude information from the dict_pos dictionary.

In [74]:
edge_x = []
edge_y = []
#las listas edges tendran las posiciones de longitud y latitud 
# Iterate over edges in the graph
for edge in G.edges():
    #print(edge)=('AER', 'KZN') o ('ASF', 'KZN') y asi
    y0, x0 = G.nodes[edge[0]]['pos']#, the code is extracting the latitude (y0) and longitude (x0) values of the first
    #node of an edge (edge[0]) from the node attributes of the graph (G).
    y1, x1 = G.nodes[edge[1]]['pos']
    #son las posisiones de los nodos de las parejas de las aristas 
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)#para un espacio
#print(edge_x)
    
# Create a Scatter trace for edges
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.25, color='#888'),
    hoverinfo='none',
    mode='lines')
#line : Defines the appearance of the lines connecting the points. It sets the line width to 0.25 and the color to a
# shade of gray ('#888').
#edge_trace is a Scatter trace designed to visualize the edges of the graph

node_x = []
node_y = []
# Iterate over nodes in the graph
for node in G.nodes():
    y, x = G.nodes[node]['pos']#latitude and longitud of the node 
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=7.5,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

Para tener en cuanta las conecciones de cada nodo

In [75]:
node_adjacencies = []
node_text = []
# Enumerate over nodes and their adjacencies in the graph
for node, adjacencies in enumerate(G.adjacency()):
    # Count the number of connections for each node
    node_adjacencies.append(len(adjacencies[1]))
    # Create a text string for the node, including the number of connections, country, and city
    node_text.append('# of connections: '+str(len(adjacencies[1]))+' '+np.array(airport_df.loc[airport_df['IATA'].isin([adjacencies[0]])]["Country"])[0]+' , '+np.array(airport_df.loc[airport_df['IATA'].isin([adjacencies[0]])]["City"])[0])

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [76]:
node_text[:5]

['# of connections: 1 Russia , Sochi',
 '# of connections: 7 Russia , Kazan',
 '# of connections: 4 Russia , Astrakhan',
 '# of connections: 1 Russia , Mineralnye Vody',
 '# of connections: 2 Russia , Chelyabinsk']

In [77]:
node_sizes = [len(list(G.neighbors(node))) for node in G.nodes()]
node_trace.marker.size = node_sizes


In [78]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph of airport routes',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=10,l=5,r=5,t=10),
                annotations=[ dict(
                    text="",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()